<a href="https://colab.research.google.com/github/Lingyi27/BME6939_HW2/blob/master/Copy_of_HW1_Almuteb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [4]:
# Import pandas
import pandas as pd
# Import numpy
import numpy as np
# Import seaborn
import seaborn as sns

#  Loading the dataset 

In [5]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


!! Your file location might be different. 




In [6]:

HW1_data = pd.read_csv('drive/My Drive/Colab Notebooks/Data/HW1/post-operative.data', names= ['L-CORE', 'L-SURF', 'L-O2', 'L-BP', 'SURF-STBL', 'CORE-STBL', 'BP-STBL',
       'COMFORT', 'Decision'])

FileNotFoundError: ignored

In [ ]:
HW1_data.describe()

In [ ]:
HW1_data['L-CORE'].unique()

In [ ]:
HW1_data.head()

In [ ]:
HW1_data.COMFORT.dtype

In [ ]:
HW1_data.describe()

In [ ]:
pd.set_option('display.max_rows', None)
print(HW1_data)


##Attribute Information:


1.   L-CORE (patient's internal temperature in C):
              high (> 37), mid (>= 36 and <= 37), low (< 36)
2. L-SURF (patient's surface temperature in C):
              high (> 36.5), mid (>= 36.5 and <= 35), low (< 35)
3. L-O2 (oxygen saturation in %):
              excellent (>= 98), good (>= 90 and < 98),
              fair (>= 80 and < 90), poor (< 80)
4. L-BP (last measurement of blood pressure):
              high (> 130/90), mid (<= 130/90 and >= 90/70), low (< 90/70)
5. SURF-STBL (stability of patient's surface temperature):
              stable, mod-stable, unstable
6. CORE-STBL (stability of patient's core temperature)
              stable, mod-stable, unstable
7. BP-STBL (stability of patient's blood pressure)
              stable, mod-stable, unstable
8. COMFORT (patient's perceived comfort at discharge, measured as
              an integer between 0 and 20)
              **  !!Attribute 8 has 3 missing values!!**
9. decision ADM-DECS (discharge decision):
              I (patient sent to Intensive Care Unit),
              S (patient prepared to go home),
              A (patient sent to general hospital floor)






## Data visualization
First we need to set all the data in order to graph.

#Convert datatype
We needed to chang the COMFORT data type from python object "O" to float 

In [ ]:
HW1_data.COMFORT.dtype

Some values can't be converted to a numeric type!
The default behaviour is to raise if it can't convert a value. In this case, it can't cope with the string '?':

```
HW1_data['COMFORT']= pd.to_numeric(HW1_data['COMFORT'])
HW1_data.COMFORT.dtype
ValueError: Unable to parse string "?" at position 46
```
Rather than fail, we  want '?' to be considered a missing/bad numeric value. We can coerce invalid values to NaN as follows using the errors keyword argumen



In [ ]:
HW1_data['COMFORT']= pd.to_numeric(HW1_data['COMFORT'], errors='coerce')
HW1_data.COMFORT.dtype

In [ ]:
HW1_data.shape

In [ ]:
HW1_data.describe()

In [ ]:
np.unique(HW1_data.Decision,return_counts = True)

Also we see that there is 4 Decision values because one A is miss input with a space. 

In [ ]:
HW1_data.Decision = np.where(HW1_data.Decision == 'A ','A',HW1_data.Decision)
np.unique(HW1_data.Decision,return_counts = True)

In [ ]:
sns.catplot(data=HW1_data, kind="violin", x="L-CORE", y="COMFORT" , hue="Decision" ) 


In [ ]:
sns.catplot(data=HW1_data, kind="violin", x="L-SURF", y="COMFORT" , hue="Decision") 


In [ ]:
sns.catplot(data=HW1_data, kind="violin", x="L-O2", y="COMFORT" , hue="Decision") 


In [ ]:
sns.catplot(data=HW1_data, kind="violin", x="L-BP", y="COMFORT" , hue="Decision") 

In [ ]:
sns.catplot(data=HW1_data, kind="violin", x="SURF-STBL", y="COMFORT" , hue="Decision") 

In [ ]:
sns.catplot(data=HW1_data, kind="violin", x="CORE-STBL", y="COMFORT" , hue="Decision") 

In [ ]:
sns.catplot(data=HW1_data, kind="violin", x="BP-STBL", y="COMFORT" , hue="Decision") 

#Preprocessing 

Mapping ordinal features
We have 'L-CORE', 'L-SURF', 'L-O2', 'L-BP', 'SURF-STBL', 'CORE-STBL', 'BP-STBL'.

In [ ]:
temp_size_mapping = {'high': 3,'mid': 2,'low': 1}
HW1_data['L-CORE'] = HW1_data['L-CORE'].map(temp_size_mapping)
HW1_data['L-SURF'] = HW1_data['L-SURF'].map(temp_size_mapping)

In [ ]:
O2_size_mapping = {'poor':1,'fair':2,'good':3,'excellent':4}
HW1_data['L-O2'] = HW1_data['L-O2'].map(O2_size_mapping)

In [ ]:
BP_size_mapping = {'high': 3,'mid': 2,'low': 1}
HW1_data['L-BP'] = HW1_data['L-BP'].map(BP_size_mapping)

In [ ]:
stbl_size_mapping = {'unstable':1,'mod-stable':2,'stable':3}
HW1_data['SURF-STBL'] = HW1_data['SURF-STBL'].map(stbl_size_mapping)
HW1_data['CORE-STBL'] = HW1_data['CORE-STBL'].map(stbl_size_mapping)
HW1_data['BP-STBL'] = HW1_data['BP-STBL'].map(stbl_size_mapping)

The categeorical variables neet to transform to one hot encoded, we have the "Decision" attribute.

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer

preprocess = make_column_transformer(
    (OneHotEncoder(), ['Decision']),
    (StandardScaler(), ['COMFORT']),
    remainder="passthrough"
)

HW1_data_fit = preprocess.fit_transform(HW1_data)
HW1_data_fit.shape
pd.DataFrame(HW1_data_fit,columns= ['','','','COMFORT', 'L-CORE', 'L-SURF', 'L-O2', 'L-BP', 'SURF-STBL', 'CORE-STBL', 'BP-STBL'])


The 3 first columns are the (discharge decision) info, and then we have the standerd form of conmfort. the rest of the columns are according tho the mappig in the previos steps, 'L-CORE', 'L-SURF', 'L-O2', 'L-BP', 'SURF-STBL', 'CORE-STBL', 'BP-STBL'. 